## 1. 导入库

In [1]:
import sklearn

In [2]:
sklearn.__version__

'0.19.2'

In [26]:
# https://blog.csdn.net/xiaosa_kun/article/details/84868437

In [1]:
import sys
sys.path.append("/Users/zhengwenjie/AI/work/ML_3/2017-CCF-BDCI-AIJudge")
import pandas as pd
from config.db_config import Config
import numpy as np
from utils import LOGGER

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import codecs

## 2.加载数据

In [99]:
config = Config()
config.data_csv_path = "/Users/zhengwenjie/AI/work/ML_3/2017-CCF-BDCI-AIJudge/data/output/corpus/data1.csv"

In [100]:
train_df = pd.read_csv(config.data_csv_path,encoding='utf-8',sep=',')
train_df = train_df.dropna()
train_df['penalty'] = train_df['penalty'].astype(int)
train_df.head()

,id,content,laws,penalty
0,16,公诉 机关 霍邱县 人民检察院 被告人 许某 1975 日生 2012 因涉嫌 危险 驾驶 ...,"133,72,73",3
1,32,公诉 机关 海口市 龙华区 人民检察院 被告人 王某 海口市 龙华区 人民检察院 海龙 检公...,"347,67,52,64",1
2,41,公诉 机关 广东省 潮州市 人民检察院 被告人 覃学彬 1980 出生 广西壮族自治区 大新...,"263,25,52,53,55,56",5
3,57,公诉 机关 榆林市 榆阳区 人民检察院 上诉人 原审 被告人 2012 因涉嫌 盗窃罪 榆林...,"264,52,53,67,72,73",5
4,60,公诉 机关 榆阳区 人民检察院 上诉人 原审 被告人 刘某 汉族 陕西省 横山县 小学文化 ...,"224,25,26,27,52,72",7


In [101]:
train_df.shape

(19, 4)

## 查看训练集每个分类的名字以及样本数量

In [102]:
# 类别：数据不均衡，需要进行采样 to do
for name,group in train_df.groupby(by='penalty'):
    print(name,len(group))

1 3
2 5
3 1
4 1
5 3
6 1
7 3
8 2


In [103]:
words_list=[]

In [104]:
for document in train_df['content']:
    words_list.append(document.split())

In [105]:
print(words_list[0:2])

[['公诉', '机关', '霍邱县', '人民检察院', '被告人', '许某', '1975', '日生', '2012', '因涉嫌', '危险', '驾驶', '霍邱县', '公安局', '取保候审', '2013', '日经', '本院', '取保候审', '霍邱县', '人民检察院', '以霍检', '刑诉', '2013', '起诉书', '指控', '被告人', '许某', '甲犯', '危险', '驾驶', '2013', '日向', '本院', '提起公诉', '本院', '依法', '简易程序', '实行', '独任', '审判', '2013', '公开', '开庭审理', '本案', '霍邱县', '人民检察院', '检察员', '胡涛', '被告人', '许某', '到庭', '参加', '诉讼', '现已', '审理', '终结', '霍邱县', '人民检察院', '指控', '2012', '被告人', '许某', '酒后', '驾驶', '二轮', '摩托车', '沿霍寿路', '由南向北', '行驶', '霍寿路', '公园路', '交叉口', '路边', '行人', '相撞', '公安民警', '查获', '六安市', '疾病', '预防', '控制中心', '鉴定', '许某', '血液', '乙醇', '含量', '169.64', 'mg', '100ml', '上述事实', '被告人', '开庭审理', '过程', '无异议', '被害人', '杨正响', '陈述', '证人', '李某', '证言', '六安市', '疾病', '预防', '控制中心', '检验', '报告', '六安市', '疾控交', '检字', '2012', '155', '霍邱县', '公安局', '交通管理', '大队', '呼吸', '酒精', '检测', '抽取', '当事人', '血样', '登记表', '驾驶', '信息', '查询', '道路', '交通事故', '赔偿', '调解', '协议书', '经济', '赔偿', '凭证', '谅解', '被告人', '户籍', '信息', '证据', '证明', '足以认定'], ['公诉', '机关', '海口市', '龙华区', '人民检察院', '被告人', '王某', '海口市

## 3. 特征工程
TfidfVectorizer模型

In [106]:
#调用sklearn.feature_extraction.text库的TfidfVectorizer方法实例化模型对象。
#TfidfVectorizer方法需要4个参数。
#第1个参数是分词结果，数据类型为列表，其中的元素也为列表；
#第2个关键字参数stop_words是停顿词，数据类型为列表；
#第3个关键字参数min_df是词频低于此值则忽略，数据类型为int或float;
#第4个关键字参数max_df是词频高于此值则忽略，数据类型为Int或float。
#查看TfidfVectorizer方法的更多参数用法，
#官方文档链接：http://sklearn.apachecn.org/cn/0.19.0/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [107]:
tfidf=TfidfVectorizer(words_list,min_df=3,max_df=0.95)

In [108]:
#第1行代码查看向量化的维数，即特征的维数；
#第2行代码调用TfidfVectorizer对象的fit_transform方法获得特征矩阵赋值给X；
#第3行代码查看特征矩阵的形状。
X=tfidf.fit_transform(train_df['content'])
print('词表大小:', len(tfidf.vocabulary_))
print(X.shape)

词表大小: 221
(19, 221)


In [109]:
#tfidf.vocabulary_

In [110]:
X.toarray()[0:4]

array([[0.        , 0.24422008, 0.38763061, 0.1406024 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.07709381,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.1406024 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.23751006, 0.        , 0.        , 0.16281339,
        0.        , 0.1406024 , 0.05955932, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.05955932, 0.        , 0.05955932,
        0.        , 0.        , 0.        , 0.05955932, 0.        ,
        0.        , 0.22089808, 0.        , 0.        , 0.        ,
        0.        , 0.1406024 , 0.08609506, 0.  

## 4. 模型训练

### 4.1 标签编码

In [111]:
np.unique(train_df['penalty'])

array([1, 2, 3, 4, 5, 6, 7, 8])

In [112]:
#调用sklearn.preprocessing库的LabelEncoder方法对文章分类做标签编码。
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_df['penalty'])

In [113]:
y.shape

(19,)

In [114]:
y

array([2, 0, 4, 4, 6, 0, 6, 1, 1, 1, 3, 7, 1, 7, 4, 1, 5, 0, 6])

### 4.2 逻辑回归模型
调用sklearn.linear_model库的LogisticRegression方法实例化模型对象。  <br>
调用sklearn.model_selection库的train_test_split方法划分训练集和测试集。

In [155]:
def micro_avg_f1(y_true, y_pred):
    '''
    分类 评估函数
    :param y_true: 样本实际类别
    :param y_pred: 预测类别
    :return:
    '''
    return f1_score(y_true, y_pred, average='micro')

In [115]:
X_train,X_test,y_train,y_test= train_test_split(X, y, test_size=0.3)

In [140]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr.fit(X_train, y_train)
y_pred = lr.predict_proba(X_test)
y_pred_score = micro_avg_f1(y_test,lr.predict(X_test))
y_pred_score

0.16666666666666666

In [141]:
y_pred

array([[0.11555073, 0.42320838, 0.10419864, 0.1586219 , 0.06712954,
        0.13129081],
       [0.16548766, 0.26835218, 0.07591945, 0.23722372, 0.08108692,
        0.17193007],
       [0.16721956, 0.24470529, 0.07614123, 0.24624113, 0.07255213,
        0.19314066],
       [0.25334315, 0.23970048, 0.07162079, 0.21381941, 0.07830657,
        0.1432096 ],
       [0.1401417 , 0.28548441, 0.07803965, 0.21018299, 0.08116869,
        0.20498256],
       [0.12379151, 0.3632005 , 0.07176894, 0.22708078, 0.06820116,
        0.14595711]])

In [142]:
lr.score(X_test,y_test)

0.16666666666666666

### 4. 3 交叉验证

In [132]:
skf = StratifiedKFold(n_splits=2, random_state=config.seed)

In [157]:
for train_index, test_index in skf.split(X_train, y_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_data,val_data=X_train[train_index],X_train[test_index]
    train_y,val_y=y_train[train_index],y_train[test_index]
    print("Train_dataset...")
    print(train_data.toarray(),train_y)
    print("Validate_dataset...")
    print(val_data.toarray(),val_y)
    lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    lr.fit(train_data, train_y)
    y_pred_val=lr.predict(val_data)
    print("y_true y_pred.....")
    print(val_y,y_pred_val)
    y_pred_score = micro_avg_f1(val_y,y_pred_val)
    y_pred_score
    print("y_pred_score={}".format(y_pred_score))

TRAIN: [ 4  6  9 10 12] TEST: [ 0  1  2  3  5  7  8 11]
Train_dataset...
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.01557146 0.26145359 0.10732344 ... 0.         0.         0.        ]
 [0.         0.         0.31856936 ... 0.         0.         0.        ]
 [0.         0.15781194 0.02683736 ... 0.01780416 0.08221929 0.        ]
 [0.         0.25561113 0.         ... 0.         0.         0.        ]] [0 7 1 4 1]
Validate_dataset...
[[0.         0.         0.55420897 ... 0.01885474 0.01741417 0.12370717]
 [0.         0.26705624 0.         ... 0.         0.38957818 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.11803844 0.0683425  0.         ... 0.05397225 0.         0.05901922]
 [0.         0.         0.         ... 0.         0.         0.16467604]
 [0.13384364 0.23248041 0.         ... 0.         0.         0.        ]] [4 1 0 1 7 5 4 3]
y_true y_pred.....
[4 1 0 1 7 5 4 3] [1 1 1 1 1 1 1 1]
y_pred_score

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)


## 4.4 基于模型的特征提取

tf-idf 提供的原始样本后，然后通过LR。。。等模型提取 特征。  通过多个模型提取特征，进行特征融合，然后再进行训练

In [160]:
num_class = len(np.unique(train_df['penalty']))
num_class

8

In [165]:
# 初始化stack ： 通过模型提取特征矩阵的数值
stack=np.zeros((X_train.shape[0],num_class))

In [166]:
for train_index, test_index in skf.split(X_train, y_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_data,val_data=X_train[train_index],X_train[test_index]
    train_y,val_y=y_train[train_index],y_train[test_index]
    print("Train_dataset...")
    print(train_data.toarray(),train_y)
    print("Validate_dataset...")
    print(val_data.toarray(),val_y)
    lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    lr.fit(train_data, train_y)
    y_pred_val=lr.predict(val_data)
    print("y_true y_pred.....")
    print(val_y,y_pred_val)
    y_pred_score = micro_avg_f1(val_y,y_pred_val)
    y_pred_score
    print("y_pred_score={}".format(y_pred_score))

TRAIN: [ 4  6  9 10 12] TEST: [ 0  1  2  3  5  7  8 11]
Train_dataset...
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.01557146 0.26145359 0.10732344 ... 0.         0.         0.        ]
 [0.         0.         0.31856936 ... 0.         0.         0.        ]
 [0.         0.15781194 0.02683736 ... 0.01780416 0.08221929 0.        ]
 [0.         0.25561113 0.         ... 0.         0.         0.        ]] [0 7 1 4 1]
Validate_dataset...
[[0.         0.         0.55420897 ... 0.01885474 0.01741417 0.12370717]
 [0.         0.26705624 0.         ... 0.         0.38957818 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.11803844 0.0683425  0.         ... 0.05397225 0.         0.05901922]
 [0.         0.         0.         ... 0.         0.         0.16467604]
 [0.13384364 0.23248041 0.         ... 0.         0.         0.        ]] [4 1 0 1 7 5 4 3]
y_true y_pred.....
[4 1 0 1 7 5 4 3] [1 1 1 1 1 1 1 1]
y_pred_score

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)


In [71]:
#调用pickle库的dump方法保存模型，需要2个参数。
#第1个参数是保存的对象，可以为任意数据类型，因为有3个模型需要保存，所以下面代码第1个参数是字典。
#第2个参数是保存的文件对象，数据类型为_io.BufferedWriter

In [72]:
import pickle

In [75]:
with open('tfidf_lr.model','wb') as f:
    save = {
        'label_encoder' : label_encoder,
        'tfidfVectorizer' : tfidf,
        'lr' : lr
    }
    pickle.dump(save, f)